In [1]:
import time
import os 
import psutil
import datetime
from matplotlib import pyplot as plt
import numpy as np
from sklearn import metrics
import pandas as pd
from scipy import stats
from IPython.display import display
import tensorflow as tf
import gpflow 

ModuleNotFoundError: No module named 'psutil'

In [ ]:
percentages = [1,2,5,10,20,30,40,50,60,70,80,90,100]
for i in percentages:
    tf.compat.v1.get_default_graph()
    tf.compat.v1.set_random_seed(0)
    tf.random.set_seed(0)
    gpflow.config.set_default_float(np.float64)
    %run ./time.py $i


In [ ]:
percentages = [2,5,10,20,30,40,50,60,70,80,90,100]
time_list = ['00:16:42','00:16:41','00:18:17','00:22:41','00:27:10','00:34:15','00:43:29','00:54:14','1:07:06','1:24:55','1:49:09','2:20:52']
minutes= []
for i in range(len(time_list)):
    x = time.strptime(time_list[i].split(',')[0],'%H:%M:%S')
    second = datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
    minutes.append(second/60.0)
    print(minutes[i])

In [ ]:
percentages = [2,5,10,20,30,40,50,60,70,80,90,100]
plt.figure(figsize=(9,5))
plt.rcParams.update({'font.size': 13})
plt.xticks(percentages)
plt.scatter(percentages,minutes,s=30,label ='time in minutes')
plt.tick_params(labelsize='large', width=2)

plt.show()

In [ ]:
GPcounts_NB_full = pd.read_csv('ll_true_100_percentage_Negative_binomial_pseudoT_Index_10.csv',index_col=[0])
GPcounts_NB_full_sorted = GPcounts_NB_full.sort_values(by=['log_likelihood_ratio'], ascending=False)

In [ ]:
percentage = np.array(list(range(1,11)))
percentage = percentage * .10 # cut a percentage from the dataset
percentages = [1,2,5,10]
M = [5,10,25,50]
counts = 0
for dataset in percentages:
    print(dataset)
    GPcounts_NB = pd.read_csv('ll_true_'+str(dataset)+'_percentage_Negative_binomial_pseudoT_Index_10.csv',index_col=[0])
    GPcounts_NB_sorted = GPcounts_NB.sort_values(by=['log_likelihood_ratio'], ascending=False)
    print(GPcounts_NB.shape)
    GPcounts_NB_index = list(GPcounts_NB_sorted.index.values)
    # index of genes ranked by log likelihood ratio from GPcounts with full NB likelihood
    GPcounts_NB_full_index = list(GPcounts_NB_full_sorted.index.values) 
    # index of genes ranked by log likelihood ratio from GPcounts with sparse NB likelihood
    GPcounts_NB_index = list(GPcounts_NB_sorted.index.values)

    corr_NB = [] # correlation score between GPcounts_NB full rank and GPcounts_NB sparse rank

    for count in range(10):
        print('Correlation score at:'+str(percentage[count])+' of the dataset:')
        # cut a percentage from the indexes
        GPcounts_NB_full_index_part= GPcounts_NB_full_index[int(len(GPcounts_NB_full_index) * .0) : int(len(GPcounts_NB_full_index) * percentage[count])]
        GPcounts_NB_rank_part = []

        for i in GPcounts_NB_full_index_part:
            # locate gene from NB full rank in GPcounts sparse rank to correlate ranking
            GPcounts_NB_rank_part.append(GPcounts_NB_index.index(i))

        GPcounts_NB_full_rank_part = list(range(0,len(GPcounts_NB_full_index_part)))

        rho, pval = stats.spearmanr(GPcounts_NB_full_rank_part,GPcounts_NB_rank_part)
        corr_NB.append(rho)
        print('NB sparse likelihood',rho)
 
    plt.scatter(percentage*100,corr_NB,s=30,label ='M='+str(M[counts])+'='+str(dataset)+'%')
    plt.tick_params(labelsize='large', width=2)
    plt.ylim(0.0,1.1)
    plt.legend(bbox_to_anchor=(.6, .0), loc='lower left', ncol=1)
    print(sum(corr_NB))
    counts = counts + 1
    #plt.legend()
plt.show()